In [1]:
import pandas as pd
from dask import dataframe as dd
from dask import delayed
from dask.distributed import Client
import numpy as np

In [9]:
client = Client(n_workers=4)

In [68]:
df = dd.read_csv('texts.csv')

In [4]:
df.tail()

,user_id,post_id,text
3288,10123382,294,Будущие мамы #кидбург
3289,10123382,295,Поздравление президента #кидбург Милане 6!
3290,10123382,297,А вот и 6!
3291,10123382,298,🌲 👼 Наш рождественский ангелок 6 лет назад 👼 🌲...
3292,10123382,299,Продается 1-комнатная квартира за 3 450 000 ру...


## Токенизация текста

In [2]:
import nltk
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation

#### Токенизация по предложениям.

Не будем принимать новую строку за новое предложение, потому что допускаем, что в посте все строки, не разделенные точкой, близко связаны по смыслу.

In [6]:
# pandarallel.initialize()

In [ ]:
df['text'] = df['text'].apply(lambda x: nltk.sent_tokenize(x, language='russian'), meta=('text')).compute()

Токенизация по словам, POS - tagging и лемматизация (коты- кот, села - село/сесть):

In [7]:
mystem = Mystem()
df['text'] = df['text'].parallel_apply(lambda x: [mystem.analyze(sent.lower()) for sent in x])

words - вложенный массив: пост[предложение[слово]]

Удалим нерусские слова, т.к. для обучения модели другим языкам имеется слишком мало данных для каждого из языков. Дефицит данных приведет к снижению точности регрессии при предсказании возраста пользователей    
  
POS- Tagging, Удаление нерусских слов, стоп-слов, и пунктуации вместе с эмодзи:

In [8]:
#nltk.download('stopwords')
rnc_to_upos = {
    'A':'ADJ',                                                                                                                                                                                                                                                                    
    'ADV':'ADV',                                                                                                                                                                                                                                                                    
    'ADVPRO':'ADV',                                                                                                                                                                                                                                                                    
    'ANUM':'ADJ',                                                                                                                                                                                                                                                                    
    'APRO':'DET',                                                                                                                                                                                                                                                                    
    'COM':'ADJ',                                                                                                                                                                                                                                                                    
    'CONJ':'SCONJ',                                                                                                                                                                                                                                                                  
    'INTJ':'INTJ',                                                                                                                                                                                                                                                                   
    'NONLEX':'X',                                                                                                                                                                                                                                                                      
    'NUM':'NUM',                                                                                                                                                                                                                                                                    
    'PART':'PART',                                                                                                                                                                                                                                                                   
    'PR':'ADP',                                                                                                                                                                                                                                                                    
    'S':'NOUN',                                                                                                                                                                                                                                                                   
    'SPRO':'PRON',                                                                                                                                                                                                                                                                   
    'UNKN':'X',                                                                                                                                                                                                                                                                      
    'V':'VERB',
}
russian_stopwords = stopwords.words("russian")

def preprocess(post):
    sents=[]
    for sent in post:
        words2=[]
        for word in sent:
            if 'analysis' in word and len(word['analysis']) > 0:
                word_s = word['analysis'][0]['lex'].lower().strip().replace('ё','е')
                pos = word["analysis"][0]["gr"].split(',')[0].split('=')[0].strip()
                if word_s not in russian_stopwords and word_s != " "\
                        and 1 < len(word_s) < 20 and word_s[0]>'а' and word_s[0]<'я':
                    # Transform RNC tags to UPOS
                    upos = rnc_to_upos[pos]
                    words2.append(word_s+'_'+upos)
#                       words2.append(word_s+'_'+pos)
        if len(words2) > 0: sents.append(words2)
    if len(sents) > 0: 
        return sents
    else: return np.nan

In [9]:
df['text']= df['text'].parallel_apply(preprocess)
df = df.dropna().reset_index(drop=True)

In [10]:
df.head()

,user_id,post_id,text
0,490298034,0,"[[хороший_ADJ, друг_NOUN, джош_NOUN, большой_A..."
1,514865204,0,[[выбирать_VERB]]
2,582801768,0,[[март_NOUN]]
3,455213471,0,"[[брат_NOUN, война_NOUN]]"
4,455213471,1,"[[каждый_DET, день_NOUN, ссориться_VERB], [каж..."


### Попробуем кластеризовать текст, представив его в виде word2vec

## Обучение модели word2vec

Для начала скачаем предобученную word2vec - модель, а потом дообучаем её на наших данных  
Предобученная модель: https://github.com/RaRe-Technologies/gensim-data/issues/3  
  
Предобработка: Корпус был лемматизирован и затеган.  
Параметры: размер вектора 300, размер окна 10

In [3]:
import gensim
from gensim.models import Word2Vec

#### На вход модели подадим документы в виде постов, т.к. предполагаем, что вся информация в посте связана. Потом можно будет подать документы в виде предложений и сравнить результаты

In [12]:
def merge_sentences(post):
    posts2=[]
    for sent in post:
        posts2+=sent
    return posts2

In [13]:
df['words'] = df['text'].parallel_apply(merge_sentences)

In [14]:
model = Word2Vec(sentences = df['words'], size=300, window=5, min_count=1, workers=4)

In [15]:
model.train(df['words'], total_examples=model.corpus_count, epochs=5)

(79176, 81945)

In [16]:
del df['words']

In [17]:
# Сохранить модель
model.save("word2vec.model")

#### Загрузим обученную модель

In [4]:
model=Word2Vec.load("word2vec.model")

Подсчитаем tf-idf для каждого слова

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
tfidf = TfidfVectorizer()

In [21]:
import math

def to_vector_tfidf(post):
    # Compute tf-idf for each word in each sentence   
    N = len(post)
    if N > 1: # Compute idf only for posts with more than 1 sentences 
        # Count number of sentences, containing each word 
        idf = {}
        for sent in post:
            for word in sent:
                if word in idf:
                    idf[word]+=1
                else: idf[word] = 1
        # Compute idf
        for word, count in idf.items():
            idf[word] = math.log10(N/float(count))
#         print(idf)
        
    # Compute tf
    sents=[]
    for sent in post:
        fq = dict.fromkeys(sent, 0)
        for word in sent:
            fq[word]+=1

        ln = len(sent)
        sent_vec = np.zeros(300)
        for word, fq in fq.items(): # Iterate though each word once to consider term frequency only once 
            tf = fq / ln
            tfidf = tf*idf[word] if N > 1 else tf # Consider idf only for posts with more than 2 sents
#             print('{}: {}'.format(word, tfidf))
            sent_vec = np.add(sent_vec, model.wv[word]*tfidf) # Multuple each vec on its tf-idf
        sents.append(sent_vec/ln)
    return sents

In [22]:
df['text_vec'] = df['text'].parallel_apply(to_vector_tfidf)

In [60]:
sents=[]
for post in df['text_vec']:
    for sent in post:
        sents.append(sent)

### K-Means Clustering

In [62]:
from sklearn.cluster import KMeans 

In [63]:
kmeans = KMeans(n_clusters=25, random_state=0).fit(sents)

In [71]:
df['text_vec']=df['text_vec'].parallel_apply(lambda post: kmeans.predict(post))

После хорошего обучения модели проверить качество кластеризации
Евклидово расстояние норм?..

In [ ]:
del sents 

In [72]:
df.head()

,user_id,post_id,text,text_vec
0,490298034,0,"[[хороший_ADJ, друг_NOUN, джош_NOUN, большой_A...",[9]
1,514865204,0,[[выбирать_VERB]],[23]
2,582801768,0,[[март_NOUN]],[5]
3,455213471,0,"[[брат_NOUN, война_NOUN]]",[17]
4,455213471,1,"[[каждый_DET, день_NOUN, ссориться_VERB], [каж...","[12, 9]"


In [8]:
client.close()

Далее проведем все те же операции в IDE, передав большие данные

## Проверим качество кластеризации

In [3]:
import pandas as pd
import ast
import collections
import numpy as np
from dask import dataframe as dd

In [4]:
pd.set_option('max_colwidth', 300)

In [36]:
df = pd.read_csv('clustered.csv', index_col=None, dtype={'mode_cluster':int})

In [5]:
def parse_clusts(clust_str):
    clusters=[]
    for cl in clust_str[1:-1].split('\n'):
        cls = [int(a) for a in cl.split(' ') if a !='']
        clusters.extend(cls)
    return clusters

In [6]:
def top_clusts(clust_list, top_n = 8):
    clust_count = dict.fromkeys(clust_list, 0)
    for clust in clust_list:
        clust_count[clust]+=1
    return sorted(clust_count.items(), key = lambda item: item[1], reverse = True)[:top_n]

In [7]:
def filter_by_clusts(clusts_str): 
    # Filter posts with less than 2 top clust count 
    clusts = parse_clusts(clusts_str)
    top = top_clusts(clusts)
    return int(top[0][0]) # DON'T FILTER. TEST!

    if top[0][1] < 2 or (len(top) > 2 and top[0][1] == top[1][1]):
        return np.nan
    else: return int(top[0][0]) # return top[0][0]
    

In [123]:
df['mode_cluster'] = df['cluster'].apply(filter_by_clusts)

In [124]:
df= df.dropna()

- Минусом пометил кластеры, без которых можно обойтись

1 - пацанские цитаты, брат за брата

2 - семья, любовь. Женские посты

3 - какой-то не русский язык

4 - физическое и ментальное здоровье

5 - "внимание"

6 и 27- кулинария + огород

7 - музыка, сериалы, новый релиз подборка, розыгрыш. Видимо, привязка к словам "отличный, новый, идея"

8 - Россия, политика, Путин. Точный кластер

9 - "красота" и синонимы + здоровье

10 - дача, выращивание

-11 - Односложные слова "важно", "нужно", "обязательно", "идеально", "солнце". Нужно исключить из Word2Vec, либо удалить кластер, либо не учитывать при обучении нейросети 

-12 - слово "бег" и синонимы. Очень редкий кластер. Слишком узкий

13 - физ. упраженения + вязание + много шума

14 - ребенок + мультики + сказки

-15 - почти пусто. Удалить кластер 

-17 - Обращение "дорогой + друг/подписчик". Обычно - розыгрыш, оч редкий кластер.

-18 - пусто

19 - фильмы и очень много шума

-20 - пусто

-21 - салатик

22 - деньги, заработок / фильмы про ограбление

-23 - пусто

-24 - пусто

25 - садоводство + замариновать + бабкины рецепты для здоровья. Кластер хороший

-26 - слово "понравиться"

27 - очень точный кластер про кулинарию

28 - некоторые фильмы. Хороший кластер

-29 - пусто

-30 - пусто

#### Не допускать в модель посты, если кол-во предложений их топового кластера == 1! 
Установить приемлемый порог количества предложений
Может, не учитывать короткие предложения или короткие посты? - уменьшим актуальность для многих юзеров

In [8]:
df2 = pd.read_csv('clustered-all.csv') 

In [9]:
df2['mode_cluster'] = df2['cluster'].apply(filter_by_clusts)

In [17]:
df2.loc[df2['mode_cluster']==14][0:20]

,user_id,post_id,text,cluster,mode_cluster


Пока что документ весит очень мало. Пока видно, что кластеризация такая же, как и при анализе 2гб векторов. рано делать выводы, т.к. предсказаны самые точные кластеры: 5 и 28 (6 и 27 в пред.) - про кулинарию

0 - сложно сказать. похоже на цитатки про жизнь

1 -

2 - слово "кулинария"

3 - детские игры/мультики. Хороший кластер

4- кулинария

-5 - кулинария + цитатки. Стоит удалить. чтобы больше влезло в кулинарию 

6 - время ("час"/"минута")

7 - 

8 - Односложные про любовь: "любить", "обнимать", "скучать"

-9 - "чувствовать". Лучше объеденить с 8-м

10 - Односложные про ум: "понимать"/"думать"/"сказать"

11 - Фразы про отношения с людьми. Хороший кластер

12 - ночь/спать и прогулка/идти. Похоже на романтику глазами женщины

13 - 

14 - 

-15 - месяца, "Санкт-Петербург", "Новый Год". Бессмыслица

16 - родство/отношение. "Коллега"/"дочь"/"подружка"/"знакомый"

17 - нерусский язык.

18 - слово "приятный"

19 - родственные связи. "брат"/"жениться"/"сын"/"дочь" и т.д. Точный кластер

20 - 

21 - "пробовать"/"полезно"/"пригодиться"

22 - наука: "менталитет", "эволюция", "наука", "логика"

23 - монстры, зомби

24 - исторические слова: "ретро", "искусство", "ренессанс", "война", "легендарный"

25 - физическое здоровье, медицина

26 - позитивная философия. "мечта", "истина", "счастье", "жизнь". Хороший кластер

27 - "внимание"/"удача". Конкурс либо просьба о поиске человека

28 - кулинария

29 - шум

Жалко, не было кластера про Россию и Путина.

Итог: Нужно на 5-6 кластеров меньше. 
Кластеризация не сильно точнее даже если передать все данные. Про Россию кластера не было, но появились про науку, семью и зомби.

Попробуем искать не моду кластеров, а изначально взвешивать все предложения в один вектор. На обучение модели кластеризации передадим один файл 2гб для скорости. Если точность будет лучше первого эксперимента, то обучить на всех данных (17 Гб).

### Результат кластеризации по постам

In [33]:
df= pd.read_csv('post-clustered.csv')

0 - музыка/трек/песня. Очень точно

1 - дети/воспитание/игры. очень чистый кластер

2 - цитаты про отношения между людьми. Часто используется слово "человек"

3 - конкурс. чистый кластер 

4 - кулинария. точно

5 - города/страны/места. точно

-6 - хз+огород. Лишний кластер

7 - слово "заметка"

-8 - лишний кластер

9 - хз

10 - нерусский

11 - "жить"

12 - "отправлять"

13 - хз

14 - Времена дня+года "утро"/"вечер"/"зима"

15 - своими руками. Вязание, маникюр, поделки

-16 = 15. прическа/маникюр, изделия

17 - "нарываться"

18 - что-то про ислам, + цитатки

19 - "внезапно"/"неожиданно"

20 - кулинария. очень точно

-21 = 20 - кулинария

22 - советы/лайфхаки: "сохрани"/"пригодится"/"не потеряй". Гуд

23 - праздиники, подарки. Гуд 

24 - какой-то (1917) год/ новый год

25 - семейное счастье. Гуд

26 - опять кулинария

27 - религия + цитаты про человееские отношения + фильмы. Нужно сузить кластер, чтобы часть ушла в фильмы

28 - фильмы. Очень хорошо 

29 - "получить достижение/бонус" в какой-то игре. Предельно точно

В итоге, лучше ли класеры описывают данные - сказать сложно. Однако, в отличие от кластеризации по предложениям, почти в каждом кластере очень много постов, значит, кластеры получились +- сбалансированными. Тем не менее, они упускают некоторые тематики, описанные предыдущим методом, но при этом добавляют свои.

Предлагаю использовать elbow - метод для определния кол-во кластеров, либо увеличить(!) кол-во кластеров и посмотреть, что вышло. Увеличим кол-во кластеров (сначала до 40), т.к. в каждом их них много данных, при этом есть кластеры, в которых данные перепутаны (их нужно раздробить).

### N_clusters = 40, vec = post

In [4]:
df = pd.read_csv('KMeans/sklearn/n40posts.csv')

In [58]:
df[df['cluster']=='[40]'][0:30]

,user_id,post_id,text,cluster


0 - мат/грубость/насилие + шум. Мб стоит сузить кластер, увеличив кол-во кластеров, чтобы избавиться от шума

1 - конкурсы. очень точно

2 - утро/вечер. Обычно фраза "доброе утро"/"добрый вечер" точно

3 - новый год/рождество. Точно

4 - кулинария. очень точно

5 - тоже кулинария. Тоже точно

6 - слова родства (мама/дочка/внук/сын). Очень точно

7 - музыка/трек/клип. очень точно

8 - сезоны (весна/зима/июнь)

9 - все подряд. Нужно раздробить

10 - все подряд. Также попали предложения про президента, Россию и экономику, +про максимальный репост. Раздробить

11 - премьеры + конкурсы и игры + очень много шума. Дробим на 3 части минимум

12 - фильмы/сериалы, преимущественно про убийства/войну, картины, книги. Хороший кластер, но, возможно стоит немножко раздробить по основным жанрам, либо в самом кластере уже классифицировать: фильм это или спектакль, какая тональность, и т.п. С помощью какой-нибудь нейронки, натренерованной на видах искусства.

13 - позитивные отзывы. круто/понравилось/красиво/вкусно/интересный. Очень точно

14 - кулинария. очень точно

15 - розыгрыши / максимальный репост. Очень точно

16 - огород, электротехника, физ. упражнения. Дает инфу о возрасте, но мб стоит раздробить

17 - хз что, дробим.

19 - "день". Обычно праздники + "добрый день", "рабочий день", "хороший день для перфекциониста" и т.д. Точно, но было бы неплохо разделить празники от обычных фраз про день.

20 - празндники + подарки. очень точно

21 - цитаты про счастье. Преимущественно позитив, иногда негатив. Очень хорошо, но было бы лучше, если бы позитив отделялся от негатива

22 - отдых/отпуск. Очень точно

23 - "сохраните, пригодится" - советы/лайфхаки

24 - фразы со словом "хороший"/"прекрасный". Часто про музыку. Несет информацию об оптимизме автора. Гуд кластер

25 - "крутой"/"шедевральный" и т.п. Фразы позитивных отзывов 

26 - продажа чего-либо. точный кластер

27 - опять кулинария (уже 4-я). точно

28 - какое-либо "настроение". Позитив. точно

29 - дружба. Точно

30 - цитаты про жизнь, заработок и др. Раздробить

31 - нерусский язык. Норм

32 - экзестенциальные быдло-цитаты. Больше негатива, чем позитива. В целом лучше раздробить на цитаты о чем-то конкретном

33 - фразы про любовь. точно

34 - разные вещи + сериалы/фильмы про любовь от 1 юзера. Дробим

35 - игры вконтакте. точно

36 - образование / школа. Очень точный и полезный кластер

37 - города/места + немного спорт. Стоит разбробить, чтобы спорт был отдельно, а также классифицировать, что за места: танц/бойцовский/компьютерный клуб.  

38 - нерусский

39 - достижение/новый уровень - игры вконтакте. точно

Стало несравненно лучше! Можно добавить столько кластеров, на сколько ещё нужно раздробить имеющиеся.

Советы по обучению нейронки:
 - Нужно посмотреть, чем кластеры кулинарии друг от друга отличаются. Может, не стоит их третировать как одно 
 - Кластеры на одинаковые темы слить в 1 общий кластер. Но будь уверен в том, что темы одинаковы
 - Можно отфильтровывать любителей преимущественно конкурсов, т.к. это могут быть фейки

Хотелось бы получить отдельный кластер про Россию и про заработок

Добавим ещё 20 кластеров и посморим, что получилось. На самом деле нужно примерно 15 кластеров, но теперь я думаю идти не "снизу вверх", а "сверху вниз"

In [18]:
len(df[df.cluster=='[48]'])

2484

In [4]:
df = pd.read_csv('KMeans/sklearn/n60posts.csv')

59 - кулинария

58 - чет смешанное

57 - своими руками, больше про вязание, но есть и про другое. Дробить не надо

56 - отзывы: классно прикольно

55 - экзестенциальные про семью, душу, жизнь, сердце, религию

54 - розыгрыши/реклама

53 - здоровье и спорт + огород и бабкины рецепты про здоровье

52 - занятия с детьми

51 - праздник+ шум. 25к

49 - хз что. надо дробить

48 - фильмы/театр/книги, иногда муз. группы + шум. 2.5к

47 - животные + "помогите человеку" + шум

46 - работа/учеба

45 - нерус

44 - праздники

43 - игры вк

42 - музыка

41 - кулинария

40 - "отправлять". 440

39 - много разного интересного, но перемешанного - 46к

38 - цитаты про людей

37 - "согласный" - 1.5к.

36 - идеи/лайфхаки. оч хорошо - 4к

35 - времена года - 7.5к

34 - возьми на заметку - 1.6к

33 - все подряд. 60к

32 - природа - 13к

31 - про "год" - 8к.

30 - купить/продать 6.7к

29 - кулинария

28 - нерусский

27 - сохрани пригодится. 4к

26 - любить. 7.5к

25 - все подряд. 210к

24 - цитаты + все подряд. 136к! Нужна отдельная кластеризация

23 - отдых. 6к

22 - красивые вещи/люди. Чаще всего прически/маникюр. 10.5к. Стоит разделить салонную красоту от остальной

21 - десерты

20 - "смысл" - 350

19 - "настроение"/"вдохновение" 1.8к

18 - "комментарий". 1 слово, 332шт.

17 - мечта счастье добро радость

16 - стихотворения + имена (поэтов? - были Пушкин и Достоевский)

15 - салаты

14 - игра вк - приз

13 - петиция. 1.3к

12 - друг/подруга. 5к

11 - места/города/страны

10 - фразы со словом "день". Чаще всего праздники, но бывают и обычные фразы. 11к

9 - про "жизнь" - 8к.

8 - цитаты с "знать"/"понимать"/"думать". Стоит разделить интеллектуалов от быдла. 36к

7 - доброе утро, спокойной ночи - 5.3к

6 - игры вк. 112к...

5 - кулинария

4 - дети

3 - розыгрыши

2 - спорт/состязания. размыто. 16к

1 - опять цитатки хз о чем - 70к.

0 - опять кулинариия - 27к.

Сразу вывод: дробим дальше и сильнее. Не хочется проверять до конца, слишком долго

Сразу вывод был поспешным...

Итог: Преимущественное большинство постов попадает под тему "цитатки" или "хз о чем". Эти огромые разделы нужно классифицировать отдельно.

Сначала будем классифицировать по этой модели, а если юзер попал в 1 из кластеров-ноунейнов, то классифицировать по второй модели, либо просто добавить центроиды из второй модели к первой.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Clustering/KMeans/rep-n200posts.csv')

In [4]:
len(df)

1412994

In [6]:
df[df['user_id']==490298034]

,user_id,post_id,text,cluster
0,490298034,0,"[['хороший_A', 'друг_S', 'джош_S', 'большой_A'...",[87]


In [4]:
df[df.cluster=='[193]']

,user_id,post_id,text,cluster
511,507233864,24,"[['способ_S', 'носить_V', 'шарф_S', 'практичес...",[193]
512,507233864,25,"[['отличный_A', 'совет_S', 'трюк_S', 'шитье_S']]",[193]
1101,578550729,3,"[['хороший_A', 'специалист_S', 'однако_CONJ', ...",[193]
1355,582295579,16,"[['памятка_S', 'смешивание_S']]",[193]
1805,556847003,29,"[['видео_S', 'очень_ADV', 'подобный_A', 'инстр...",[193]
1819,323751271,41,"[['клумба_S', 'цветник_S', 'шина_S', 'свой_APR...",[193]
2705,278360652,60,"[['хитрость_S', 'настоящий_A', 'фанат_S', 'чис...",[193]
2770,398631996,12,"[['гид_S', 'начинающий_A', 'сомелье_S', 'сохра...",[193]
2815,398631996,37,"[['совет_S', 'консервация_S']]",[193]
3256,156159818,7,"[['статус_S', 'цитата_S', 'стих_S', 'рецепт_S'...",[193]


Только интересные:

0 - места развлечений (клуб/цирк/фестиваль). 2.5к

1 - здоровье и спорт. 11к

2 - дороги/машины

8 - петиция - 1к

12 - еда на ужин

13 - небо солнце свет. 6к

14 - 34к цитаток

18 = 19. 2к и 1.8к

20 - 30к цитаток

29 - смысл/истина. 2к

33 - цитаты про мужчин и женщин. 7.6к

35 - искусство. 3.8к

37 - новости, политика, Россия. 5.8к

43 - 90к мешанины

47 - 125к цитат

50 - 22к мещанины

51 - 31к всего подряд

55 - физ. упражнения

60 - 26к смешанного

64 - дача

68 - спорт по телевизору - 3.5к

77 - девочки - 1.5к

79 - вязание. 4к

83 - православие. 3к

100 - находить/пропадать - про людей и вещи. 21210к

106 - классика. 250

118 - учеба - 7к.

124 - рисование

129 - подборки (топ). 2к

130 - наука/история 

137 - 95к смешанного

138 - органы РФ, Россия. 13к

145 - маникюр/макияж. Теперь отдельно от причесок. 4к

Обязательно должны быть кластеры, описанные в предыдущих опытах + кластер ватников, экономики, интеллектуалов.

Думаю, дальнейшую точность кластеров будет вычислять по точности предсказаний нейронки с фиксированными параметрами
________
Стало появляться много одинаковых кластеров, при этом успешно дробятся старые и появляются новые - информативные. Некоторые передробились, но их можно +-безболезненно удалить вручную.


 - Некоторые бесполезные кластеры стоит просто удалить, чтобы алгоритм искал, куда бы ещё их добавить. Например, 134 кластер про "минутку": пусть лучше предложение "минутка поэзии" отнесется к поэзии, а не к минутке.

Удалить кластеры про :

 -"жизнь"

-"год", не относящийся к новому году.

...

________
По предобработке (видимо, уже на будущее):
 - Праздники оставить одной фразой: 8_марта, 23_февраля. Мб просто не надо было удалять числа.
 - Новый_год - одной фразой
 - слово "это" - в стоп-слова
 - не удалять нерусские слова. Пускай обучается другим языкам тоже, в этом нет ничего плохого
________
Осталось примерно 6 крупных смешанных кластеров. Предлагаю (1 из 3):

1) n_clust = 300. Потом удаляем лишние и долго маппим одинаковые кластеры

2) Кластеризовать эти крупные 6 кластеров отдельно.

3) Hierarchical clustering с послудующей классификацией knn. Перед классификацией нужно убедиться в том, что кластеризация успешная - как? Knn вроде хранит все точки, значит, придется ограничиться только 2 гб векторов, чтобы влезло в RAM. Лучше посмотри, можно ли экпортнуть центроиды из иерархичной кластеризации. Либо используй эту кластеризацию, чтобы понять кол-во кластеров для k-means

4) Обучить уже нейронку, пусть хоть какой-то результат выдает. Сроки горят.


In [4]:
df = pd.read_csv('KMeans/sklearn/n100posts.csv')

0 - сон - 7к

1 - фильмы - 20к

2 - огород, бабкины рецепты - 14к

3 - цитатки - 20к.

Теперь будем записывать только интересные кластеры

83 - школа/образование + достижения в играх.

91 - история/искусство. круто. 7к

90 - купи продай

86 - уменьшительно-ласкательные

84 - красивые прически/туфли/маникюр. 4.5к

83 - изучение, саморазвитие. 10к

82 - школа, учиться - 2.5к

80 - поделки

77 - физ. упражнения. 4к

71 - успех, огород, здоровье. Надо дробить

68 - казино

67 - семья

62 - кот/собака

61 - со словом "это". Надо удалить, а "это" добавить в стоп-слова

55 - 113к говна

38 - прикол/юмор

12 - 190к говна.


Итог: Стало больше интересных кластеров, но кластеры также стали чаще повторяться.
Попоробовать 200 и 300 (Kosinski вроде 300 брал) кластеров. Главное - раздробить оромные кластеры и выудить более подробную информацию. Если при этом много кластеров будут повторяться, надо будет их замаппить в один.
Сейчас появились хорошие кластеры про искусство и образование, + красота разделилась на женский салон и остальное. Посмотрим, что будет дальше.

_______
Мб стоит уже учить нейронку и смотреть на переобучение. Чем больше кластеров - тем больше риск переобучения

In [1]:
from dask import dataframe as dd

In [2]:
df = dd.read_parquet('Clustering/KMeans/n200posts.parquet')

In [3]:
user =df[df.user_id==455213471].compute()

In [12]:
user[user.cluster==190]

,user_id,post_id,text,type,date,cluster
index,,,,,,


## Формирование норамализованного вектора кластеров для каждого пользователя

In [4]:
user_clusts = df.groupby('user_id')['cluster'].apply(list).reset_index(name='clusters')

In [5]:
user_clusts[:10]

,user_id,clusters
0,43906,"[[43], [137], [63], [43], [63], [20], [47], [176], [69], [91], [47], [47], [25], [47], [137], [47], [63], [87], [175], [137], [137], [63], [95], [118], [87], [43], [137], [137], [63], [137], [43], [43], [47], [47], [63], [137], [137], [20], [175], [47], [137], [51], [137], [137], [137], [43], [4..."
1,80008,"[[20], [91], [24]]"
2,89278,"[[51], [116], [47], [170], [20], [170], [125], [37], [73], [68], [91], [172], [187], [159], [35], [159], [163], [43], [116], [98], [50], [50], [5], [50], [137], [20], [20], [50], [170], [163], [87], [137], [163], [50], [14], [170], [50], [51], [91], [168], [20], [170], [50], [50], [163], [26], [..."
3,99945,"[[82], [47], [63], [159]]"
4,115116,"[[43], [26], [30], [47], [47], [50], [20], [20], [163], [60], [50], [76], [47], [47], [47], [95], [52], [137], [43], [47], [50], [95], [60], [57], [31], [57], [60], [137], [51], [60], [82], [68], [198], [14], [166], [138], [63], [168], [47], [187], [57], [20], [137], [47], [47], [137], [47], [13..."
5,154271,"[[51], [69], [84]]"
6,162377,"[[37], [47], [47], [82], [137], [1], [47], [9], [134], [43], [43], [43], [138], [48], [47], [43], [137], [137], [3], [47], [13], [69], [47], [47], [91], [47], [36], [47], [43], [3], [47], [42], [116], [87], [43], [63], [63], [63], [47], [43], [63], [63], [47], [47], [3], [25], [13], [137], [137]..."
7,162413,"[[47], [60], [47], [59], [51], [47], [47], [173], [138], [59], [59], [69], [69], [112], [43], [69], [43], [69], [51], [47], [51], [100], [100], [128], [137], [59], [59], [47], [69], [69], [47], [59], [47], [43], [51], [20], [59], [31], [112], [43], [43], [112], [47], [100], [47], [47], [112], [1..."
8,192933,"[[43], [13], [151], [51], [1], [175], [51], [59], [153], [47], [47], [139], [23], [163], [175], [144], [77], [20], [100], [51], [59], [47], [191], [100], [43], [137], [172], [63], [43], [43], [51], [135], [116], [47], [87], [25], [69], [43], [48], [47], [43], [100], [31], [43], [100], [43], [47]..."
9,196756,"[[157], [163]]"


In [6]:
for i in range (200):
    user_clusts[i] = 0

In [7]:
user_clusts.head()

,user_id,clusters,0,1,2,3,4,5,6,7,...,190,191,192,193,194,195,196,197,198,199
0,43906,"[[43], [137], [63], [43], [63], [20], [47], [176], [69], [91], [47], [47], [25], [47], [137], [47], [63], [87], [175], [137], [137], [63], [95], [118], [87], [43], [137], [137], [63], [137], [43], [43], [47], [47], [63], [137], [137], [20], [175], [47], [137], [51], [137], [137], [137], [43], [4...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80008,"[[20], [91], [24]]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,89278,"[[51], [116], [47], [170], [20], [170], [125], [37], [73], [68], [91], [172], [187], [159], [35], [159], [163], [43], [116], [98], [50], [50], [5], [50], [137], [20], [20], [50], [170], [163], [87], [137], [163], [50], [14], [170], [50], [51], [91], [168], [20], [170], [50], [50], [163], [26], [...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,99945,"[[82], [47], [63], [159]]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,115116,"[[43], [26], [30], [47], [47], [50], [20], [20], [163], [60], [50], [76], [47], [47], [47], [95], [52], [137], [43], [47], [50], [95], [60], [57], [31], [57], [60], [137], [51], [60], [82], [68], [198], [14], [166], [138], [63], [168], [47], [187], [57], [20], [137], [47], [47], [137], [47], [13...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for idx, row in user_clusts.iterrows():
    for cl in row['clusters']:
        cl = int(cl[1:-1]) # Remove brackets 
        row[cl]+=1
    
    user_clusts[idx] = row